In [ ]:
# Импортирование модулей
import albumentations as A
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def synt_gereration(real_img_fullpath, real_msk_fullpath, cfg):
    '''
    Генерирует изображение с дополнительными пятнами.
    Parameters
    ----------
    real_img_fullpath : str
        Путь к папке с набором картинок.
    real_msk_fullpath : str
        Путь к папке с набором масок.
    cfg : dict
        Настройка параметров генерации.

    Returns
    -------
    synt_img : numpy.ndarray
        Сгенерированное изображение.
    synt_msk : numpy.ndarray
        Маска по сгенерированному изображению.
    '''
    img = cv2.imread(real_img_fullpath)[:,:,::-1]
    msk = cv2.imread(real_msk_fullpath)[:,:,::-1]
    (h, w, c) = img.shape
    
    # main synt generation
    List_of_Drops, label_map  = generate_label(h, w, cfg)
    synt_img, _, synt_msk = generateDrops(real_img_fullpath, cfg, List_of_Drops)
    
    # pill-image to numpy-image
    synt_img = np.array(synt_img)
    synt_msk = np.array(synt_msk)
    
    # почему-то иногда генерируется маска (h,w,3), но в основном - (h,w)
    if len(synt_msk.shape)==3: 
        synt_msk = synt_msk[:,:,0]
    synt_msk = np.stack((synt_msk,)*3, axis=-1)
    
    # совмещение реальной маски и синтетической
    # (чтобы синтетику можно было генерить и на грязных кадрах)
    synt_msk = (((msk>0).astype(int)+(synt_msk>0).astype(int))>0).astype(np.uint8)*255

    return synt_img, synt_msk

In [ ]:
# Пути
real_img_path = 'train_dataset/cv_open_dataset/open_img/'
real_msk_path = 'train_dataset/cv_open_dataset/open_msk/'
res_img_path = 'train_dataset/augmentations/augm_img/'
res_msk_path = 'train_dataset/augmentations/augm_msk/'
imgs = os.listdir(real_img_path)

In [ ]:
# Преобразования
augm_crop = A.Compose([
    A.RandomCrop(width=640, height=360, p=1),
    A.ToGray(p=0.1),
    A.MultiplicativeNoise()
])
augm_rot = A.Compose([
    A.Rotate(p=1),
    A.MultiplicativeNoise(),
    A.ToGray(p=0.1)
])
augm_hflip = A.Compose([
    A.HorizontalFlip(p=1),
    A.MultiplicativeNoise(),
    A.RandomBrightnessContrast(brightness_limit=0.1)
])
augm_vflip = A.Compose([
    A.VerticalFlip(p=1),
    A.MultiplicativeNoise(),
    A.RandomBrightnessContrast(brightness_limit=0.1)
])

In [ ]:
# Пример
img = cv2.imread(real_img_path+imgs[0])[:,:,::-1]
msk = cv2.imread(real_msk_path+msks[0])[:,:,::-1]
res = augm_crop(image=img, mask=msk)
plt.figure(figsize=(10,10))
plt.imshow(np.concatenate([res['image'], res['mask']], axis=1))

In [ ]:
# Генерация аугментированных изображений
for i in range(len(imgs)):

    print(i,end=' ')
    img = cv2.imread(real_img_path+imgs[i])[:,:,::-1]
    msk = cv2.imread(real_msk_path+imgs[i][:-4]+'.png')[:,:,::-1]

    for n in range(5):
        res = augm_crop(image=img, mask=msk)
        cv2.imwrite(res_img_path+'crop'+str(n)+'_'+imgs[i], res['image'])
        cv2.imwrite(res_msk_path+'crop'+str(n)+'_'+imgs[i][:-4]+'.png', res['mask'])

        res = augm_rot(image=img, mask=msk)
        cv2.imwrite(res_img_path+'rot'+str(n)+'_'+imgs[i], res['image'])
        cv2.imwrite(res_msk_path+'rot'+str(n)+'_'+imgs[i][:-4]+'.png', res['mask'])

        res = augm_hflip(image=img, mask=msk)
        cv2.imwrite(res_img_path+'hflip'+str(n)+'_'+imgs[i], res['image'])
        cv2.imwrite(res_msk_path+'hflip'+str(n)+'_'+imgs[i][:-4]+'.png', res['mask'])

        res = augm_vflip(image=img, mask=msk)
        cv2.imwrite(res_img_path+'vflip'+str(n)+'_'+imgs[i], res['image'])
        cv2.imwrite(res_msk_path+'vflip'+str(n)+'_'+imgs[i][:-4]+'.png', res['mask'])


In [ ]:
# Импортирование raindrop
from raindrop.config import cfg
from raindrop.dropgenerator import generate_label, generateDrops

In [ ]:
syaugm_img_path = 'train_dataset/augm_synt/augm_img/'
syaugm_msk_path = 'train_dataset/augm_synt/augm_msk/'
cfg['maxR'] = 100
augms = os.listdir(res_img_path)

In [ ]:
# Генерация пятен на изображениях с аугментациями
for i in range(len(augms)):

    print(i, end=' ')
    img = cv2.imread(res_img_path+augms[i])[:,:,::-1]
    msk = cv2.imread(res_msk_path+augms[i][:-4]+'.png')[:,:,::-1]

    synt_img,synt_msk = synt_gereration(res_img_path+augms[i], res_msk_path+augms[i][:-4]+'.png', cfg)
    cv2.imwrite(syaugm_img_path+'sy'+augms[i], synt_img[:,:,::-1])
    cv2.imwrite(syaugm_msk_path+'sy'+augms[i][:-4]+'.png', synt_msk[:,:,::-1])